In [1]:
import biosteam as bst
import thermosteam as tmo

In [2]:
from biosteam import settings

In [3]:
from biosteam import Unit, Stream, settings, main_flowsheet
import biosteam as bst
bst.nbtutorial() # Light-mode html diagrams and filter warnings
settings.set_thermo(['Water'])
ins = Stream('in0')
outs = [Stream('out0')]
U1 = Unit(ID='U1', ins=ins, outs=outs)
U1.show(data=False) # Passing data as False returns only stream names

Unit: U1
ins...
[0] in0  
outs...
[0] out0  


In [4]:
bst.settings.set_thermo(['Water', 'He','CO2','Cellulose','Hemicellulose','Lignin','Ash','Glucose','Xylose','Arabinose','Sucrose','Fructose','Furfural','Acetic Acid','Formic Acid','H2SO4'],db='BioSTEAM')

In [ ]:
from biosteam import units
import numpy as np

bst.main_flowsheet.set_flowsheet('sugarcane_ethanol')

# We can create streams and set component splits faster by defining chemical groups
chemicals = bst.settings.chemicals
chemicals.define_group(
    name='Fiber',
    IDs=['Cellulose', 'Hemicellulose', 'Lignin'],
    composition=[0.4704 , 0.2775, 0.2520],
    wt=True, # Composition is given as weight
)
chemicals.define_group(
    name='Sugar',
    IDs=['Sucrose', 'Glucose'],
    # Default composition as equimolar
)

sugarcane = bst.Stream(
    'sugarcane',
    Water=0.7,
    Glucose=0.01208,
    Sucrose=0.1369,
    Ash=0.006,
    Fiber=0.13,
    Solids=0.015,
    total_flow=370396,
    units='kg/hr',
    price=price['Sugar cane']
)

enzyme = bst.Stream('enzyme',
                    Cellulose=100, Water=900, units='kg/hr',
                    price=price['Protease'])

imbibition_water = bst.Stream('imbibition_water',
                              Water=87023.35, units='kg/hr',
                              T = 338.15)

H3PO4 = bst.Stream('H3PO4',
                   H3PO4=74.23, Water=13.10, units='kg/hr',
                   price=price['H3PO4'])  # to T203

lime = bst.Stream('lime',
                  CaO=333.00, Water=2200.00, units='kg/hr',
                  price=price['Lime'])  # to P5

polymer = bst.Stream('polymer',
                     Flocculant=0.83, units='kg/hr',
                     price=price['Polymer'])  # to T205

rvf_wash_water = bst.Stream('rvf_wash_water',
                            Water=16770, units='kg/hr',
                            T=363.15)  # to C202

### Unit operations ###

# Feed from storage
U101 = units.ConveyingBelt('U101', sugarcane)

# Separate metals
U102 = units.MagneticSeparator('U102', U101-0)

# Shredded cane
U103 = units.Shredder('U103', U102-0)

# Hydrolyze starch
T201 = units.EnzymeTreatment('T201', [U103-0, enzyme], T=323.15)  # T=50

# Finely crush lipid cane
imbibition_water_recycle = bst.Stream() # To connect later
U201 = units.CrushingMill('U201', [T201-0, imbibition_water_recycle],
                          split=dict(Ash=0.92,
                                     Fiber=0.92,
                                     Sugar=0.04,
                                     Solids=1),
                          moisture_content=0.5)

# Convey out bagasse
U202 = units.ConveyingBelt('U202', U201-0, outs='Bagasse')

# Screen out fibers
S201 = units.VibratingScreen('S201', U201-1,
                             split=dict(Ash=0.35,
                                        Fiber=0.35,
                                        Sugar=0.88,
                                        Water=0.88,
                                        Solids=0))

# Mix in water
M201 = units.Mixer('M201', [S201-1, imbibition_water], imbibition_water_recycle)

# Store juice before treatment
T202 = units.StorageTank('T202', S201-0, tau=4, vessel_material='Carbon steel')

# Heat up before adding acid
H201 = units.HXutility('H201', T202-0, T=343.15)

# Mix in acid
T203 = units.MixTank('T203', [H201-0, H3PO4])

# Pump acid solution
P201 = units.Pump('P201', T203-0)

# Mix lime solution
T204 = units.MixTank('T204', [P201-0, lime], tau=0.10)

# Blend acid lipid solution with lime
T205 = units.MixTank('T205', T204-0, tau=0.10)
P202 = units.Pump('P202', T205-0)

# Mix recycle
RVF_recycle = bst.Stream() # From rotary vacuum filter; connect later
M202 = units.Mixer('M202', [P202-0, RVF_recycle])

# Heat before adding flocculant
H202 = units.HXutility('H202', M202-0, T=372.15)

# Mix in flocculant
T206 = units.MixTank('T206', [H202-0, polymer])
T206.tau = 0.10

# Separate residual solids
C201 = units.Clarifier('C201', T206-0,
                       split=dict(Ash=0,
                                  CaO=0,
                                  Fiber=0,
                                  Flocculant=0.522,
                                  Sugar=0.522,
                                  H3PO4=0.522,
                                  Water=0.522))

# Remove solids as filter cake
C202 = units.RVF('C202', [C201-1, rvf_wash_water],
                 outs=('filter_cake', ''),
                 moisture_content=0.80,
                 split=dict(Ash=0.85,
                            CaO=0.85,
                            Fiber=0.85,
                            Sugar=0.01))
P203 = units.Pump('P203', C202-1, RVF_recycle)


# Screen out small fibers from sugar stream
S202 = units.VibratingScreen('S202', C201-0,
                             outs=('', 'fiber_fines'),
                             split=dict(Ash=1.0,
                                        CaO=1.0,
                                        Fiber=1.0,
                                        Flocculant=0.0,
                                        Sugar=0.998,
                                        H3PO4=1.0,
                                        Water=0.998))
S202.mesh_opening = 2

### Process specifications ###

# Specifications dependent on lipid cane flow rate
@U103.add_specification(run=True) # Run unit operation afterwords
def correct_flows():
    feedstock = U101.ins[0]
    F_mass = feedstock.F_mass
    # correct enzyme, lime, phosphoric acid, and imbibition water
    enzyme.imass['Cellulose', 'Water'] = 0.003 * F_mass * np.array([0.1, 0.9])
    lime.imass['CaO', 'Water'] = 0.001 * F_mass * np.array([0.046, 0.954])
    H3PO4.imass['H3PO4', 'Water'] = 0.00025 * F_mass
    imbibition_water.imass['Water'] = 0.25* F_mass

# Specifications within a system
@P202.add_specification(run=True)
def correct_wash_water():
    solids = P202.ins[0].imol['Ash', 'CaO', 'Fiber'].sum()
    rvf_wash_water.imol['Water'] = 0.0574 * solids

bst.main_flowsheet.diagram(format='png') # Flow sheet up until now

In [9]:
# example for 1 kg feed basis (adjust as needed)
bagasse_slurry = bst.Stream(
    'bagasse_slurry',
    Water=0.88,           # majority is water (slurry), set correct kg/hr
    Cellulose=0.03617,    # (36.17 wt% of dry biomass; scale to slurry basis)
    Hemicellulose=0.01968,
    Lignin=0.01766,
    Ash=0.00459,
    Sucrose=0.01155*0.1605,  # optional split: paper quotes soluble sugar breakdown; adjust
    Glucose=0.01155*0.4937,
    Fructose=0.01155*0.3458,
    units='kg/hr'
)


UndefinedChemicalAlias: 'Cellulose'

In [5]:
from biosteam import Reaction, ParallelReaction

# example Reaction objects — syntax depends on your biosteam version
r1 = Reaction('Hemicellulose -> Xylose', 'Hemicellulose', 0.941)
r2 = Reaction('Cellulose -> Glucose', 'Cellulose', 0.492)
# secondary decomposition (fractions relative to sugars present)
#r3 = Reaction('Xylose -> 0.25 Furfural + Acetic Acid + 0.10 Formic Acid', 'Xylose', 0.12)
r4 = Reaction('Glucose -> Acetic Acid + Formic Acid', 'Glucose', 0.08)
# combine into reactor
#HTL_reactor.add_reactions([r1, r2, r3, r4])


UndefinedChemicalAlias: 'AceticAcid'

In [6]:
import thermosteam as tmo
chemicals = tmo.Chemicals(['H2O', 'H2', 'O2'], cache=True)
tmo.settings.set_thermo(chemicals)
reaction = tmo.Reaction('2H2O,l -> 2H2,g + O2,g', reactant='H2O', X=0.7)
reaction.show()

Reaction (by mol):
stoichiometry             reactant    X[%]
H2O,l -> H2,g + 0.5 O2,g  H2O,l      70.00


In [7]:
import thermosteam as tmo

# example Reaction objects — syntax depends on your biosteam version
r1 = Reaction('Hemicellulose -> Xylose', 'Hemicellulose', 0.941)
r2 = Reaction('Cellulose -> Glucose', 'Cellulose', 0.492)
# secondary decomposition (fractions relative to sugars present)
r3 = Reaction('Xylose -> 0.25 Furfural + Acetic Acid + 0.10 Formic Acid', 'Xylose', 0.12)
r4 = Reaction('Glucose -> Acetic Acid + Formic Acid', 'Glucose', 0.08)
# combine into reactor
#HTL_reactor.add_reactions([r1, r2, r3, r4])

UndefinedChemicalAlias: 'Hemicellulose'